**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jan 26 21:49:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'  'My Drive'   yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 14.02 MiB | 24.58 MiB/s, done.
Resolving deltas: 100% (10353/10353), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
# !cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp "/mydrive/yolov3/yolov3_training.cfg" cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "prohibitory\ndanger\nmandatory\nother" > data/obj.names
!echo -e 'classes= 4\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-01-26 21:50:27--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  45.8MB/s    in 3.7s    

2022-01-26 21:50:31 (41.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/00899.jpg      
  inflating: data/obj/00899.txt      
  inflating: data/obj/00000.jpg      
  inflating: data/obj/00000.txt      
  inflating: data/obj/00001.jpg      
  inflating: data/obj/00001.txt      
  inflating: data/obj/00002.jpg      
  inflating: data/obj/00002.txt      
  inflating: data/obj/00003.jpg      
  inflating: data/obj/00003.txt      
  inflating: data/obj/00004.jpg      
  inflating: data/obj/00004.txt      
  inflating: data/obj/00005.jpg      
  inflating: data/obj/00005.txt      
  inflating: data/obj/00006.jpg      
  inflating: data/obj/00006.txt      
  inflating: data/obj/00007.jpg      
  inflating: data/obj/00007.txt      
  inflating: data/obj/00008.jpg      
  inflating: data/obj/00008.txt      
  inflating: data/obj/00009.jpg      
  inflating: data/obj/00009.txt      
  inflating: data/obj/00010.jpg      
  inflating: data/obj/00010.txt      
  inflating: data/obj/00011.jpg      
  inflating: 

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/00488.jpg', 'data/obj/00472.jpg', 'data/obj/00885.jpg', 'data/obj/00531.jpg', 'data/obj/00051.jpg', 'data/obj/00865.jpg', 'data/obj/00468.jpg', 'data/obj/00644.jpg', 'data/obj/00502.jpg', 'data/obj/00760.jpg', 'data/obj/00744.jpg', 'data/obj/00431.jpg', 'data/obj/00132.jpg', 'data/obj/00368.jpg', 'data/obj/00876.jpg', 'data/obj/00628.jpg', 'data/obj/00310.jpg', 'data/obj/00029.jpg', 'data/obj/00303.jpg', 'data/obj/00475.jpg', 'data/obj/00460.jpg', 'data/obj/00602.jpg', 'data/obj/00205.jpg', 'data/obj/00075.jpg', 'data/obj/00679.jpg', 'data/obj/00692.jpg', 'data/obj/00128.jpg', 'data/obj/00774.jpg', 'data/obj/00316.jpg', 'data/obj/00200.jpg', 'data/obj/00096.jpg', 'data/obj/00250.jpg', 'data/obj/00217.jpg', 'data/obj/00662.jpg', 'data/obj/00829.jpg', 'data/obj/00779.jpg', 'data/obj/00608.jpg', 'data/obj/00426.jpg', 'data/obj/00651.jpg', 'data/obj/00255.jpg', 'data/obj/00288.jpg', 'data/obj/00320.jpg', 'data/obj/00667.jpg', 'data/obj/00499.jpg', 'data/obj/00150.jpg', 'data/obj

In [ ]:
# Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# Resume training from last checkpoint
!./darknet detector train data/obj.data cfg/yolov3_training.cfg  /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 39851, rewritten_bbox = 0.130486 % 
]2;7926/8000: loss=0.0 hours left=0.1
 7926: 0.039111, 0.058408 avg loss, 0.000010 rate, 5.032620 seconds, 253632 images, 0.144698 hours left
Loaded: 0.000041 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.861994), count: 6, class_loss = 0.000028, iou_loss = 0.138067, total_loss = 0.138095 
 total_bbox = 39857, rewritten_bbox = 0.130466 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse